In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [3]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')
    dates_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))

In [4]:
#Create a pool of data
train_pool_b = Pool(X_train_new_b, y_train_new_b)

test_pool_b = Pool(X_test_new_b) 

In [5]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
    }

    catboost_model_b = CatBoostRegressor(**params, verbose=100)
    catboost_model_b.fit(train_pool_b)
    pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
    MAE_b = mean_absolute_error(y_test_b, pred_b)
    return MAE_b

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-07 08:23:13,544] A new study created in memory with name: no-name-794aae06-6f4c-4edf-af1a-81d272fc9475


0:	learn: 183.1410752	total: 232ms	remaining: 6m 22s
100:	learn: 93.5156084	total: 10.3s	remaining: 2m 37s
200:	learn: 72.7456700	total: 18.8s	remaining: 2m 15s
300:	learn: 64.3440387	total: 26.6s	remaining: 1m 59s
400:	learn: 58.7426437	total: 34.1s	remaining: 1m 46s
500:	learn: 54.4732511	total: 41.7s	remaining: 1m 35s
600:	learn: 50.8627528	total: 53.2s	remaining: 1m 32s
700:	learn: 47.7325945	total: 1m 4s	remaining: 1m 27s
800:	learn: 44.8904605	total: 1m 16s	remaining: 1m 21s
900:	learn: 42.2077479	total: 1m 28s	remaining: 1m 13s
1000:	learn: 39.8561935	total: 1m 40s	remaining: 1m 5s
1100:	learn: 37.7530768	total: 1m 53s	remaining: 56.6s
1200:	learn: 35.7878693	total: 2m 4s	remaining: 46.4s
1300:	learn: 33.9199214	total: 2m 15s	remaining: 36.2s
1400:	learn: 32.2936308	total: 2m 25s	remaining: 25.9s
1500:	learn: 30.8373477	total: 2m 36s	remaining: 15.6s
1600:	learn: 29.4873476	total: 2m 47s	remaining: 5.14s
1649:	learn: 28.8550334	total: 2m 53s	remaining: 0us


[I 2023-11-07 08:26:09,481] Trial 0 finished with value: 84.12810567645126 and parameters: {'iterations': 1650, 'learning_rate': 0.014468942204644675, 'depth': 13, 'min_data_in_leaf': 96, 'l2_leaf_reg': 4, 'bagging_temperature': 0.44085895917453866, 'random_strength': 0.6383914875970905, 'border_count': 152, 'rsm': 0.210952114395627}. Best is trial 0 with value: 84.12810567645126.


0:	learn: 184.7001107	total: 5.72ms	remaining: 2.79s
100:	learn: 158.8122580	total: 504ms	remaining: 1.94s
200:	learn: 141.0194853	total: 1.08s	remaining: 1.55s
300:	learn: 126.8833882	total: 1.64s	remaining: 1.02s
400:	learn: 116.4534296	total: 2.13s	remaining: 467ms


[I 2023-11-07 08:26:12,227] Trial 1 finished with value: 128.13063357628448 and parameters: {'iterations': 489, 'learning_rate': 0.0031025345542633684, 'depth': 5, 'min_data_in_leaf': 8, 'l2_leaf_reg': 10, 'bagging_temperature': 0.3273842036865815, 'random_strength': 0.3667279287241337, 'border_count': 44, 'rsm': 0.06515251078017643}. Best is trial 0 with value: 84.12810567645126.


...

0:	learn: 183.5544393	total: 3.02ms	remaining: 6.87s
100:	learn: 111.6508423	total: 823ms	remaining: 17.7s
200:	learn: 96.1110316	total: 1.69s	remaining: 17.4s
300:	learn: 91.6656101	total: 2.81s	remaining: 18.4s
400:	learn: 89.5704192	total: 3.55s	remaining: 16.6s
500:	learn: 88.0580494	total: 4.24s	remaining: 15s
600:	learn: 86.8723236	total: 5s	remaining: 13.9s
700:	learn: 85.8878262	total: 5.79s	remaining: 13s
800:	learn: 85.0610827	total: 6.61s	remaining: 12.1s
900:	learn: 84.2925952	total: 7.3s	remaining: 11.1s
1000:	learn: 83.5000658	total: 8.28s	remaining: 10.5s
1100:	learn: 82.8668697	total: 8.98s	remaining: 9.54s
1200:	learn: 82.3215820	total: 9.95s	remaining: 8.85s
1300:	learn: 81.8108189	total: 10.7s	remaining: 7.96s
1400:	learn: 81.3853085	total: 11.5s	remaining: 7.16s
1500:	learn: 81.0209125	total: 12.5s	remaining: 6.4s
1600:	learn: 80.6507676	total: 13.5s	remaining: 5.64s
1700:	learn: 80.2923107	total: 14s	remaining: 4.7s
1800:	learn: 79.9663899	total: 14.6s	remaining: 3

[I 2023-11-07 09:12:31,436] Trial 78 finished with value: 74.3588153681617 and parameters: {'iterations': 2270, 'learning_rate': 0.012094035387133772, 'depth': 3, 'min_data_in_leaf': 32, 'l2_leaf_reg': 8, 'bagging_temperature': 0.9071854056711158, 'random_strength': 0.6733358153014276, 'border_count': 94, 'rsm': 0.33204814102977553}. Best is trial 55 with value: 72.65112909096902.


2269:	learn: 78.6354849	total: 17.9s	remaining: 0us
0:	learn: 183.9126231	total: 9.97ms	remaining: 19.4s
100:	learn: 114.8573902	total: 1.82s	remaining: 33.4s
200:	learn: 92.0655821	total: 3.97s	remaining: 34.5s
300:	learn: 84.0212566	total: 5.98s	remaining: 32.7s
400:	learn: 80.1257096	total: 8.51s	remaining: 32.8s
500:	learn: 77.6587195	total: 10.9s	remaining: 31.3s
600:	learn: 75.7713953	total: 12.5s	remaining: 28s
700:	learn: 74.0607471	total: 14.3s	remaining: 25.4s
800:	learn: 72.5636609	total: 16.1s	remaining: 23.1s
900:	learn: 71.1494403	total: 17.8s	remaining: 20.7s
1000:	learn: 69.8575988	total: 20.3s	remaining: 19.2s
1100:	learn: 68.7757568	total: 22.4s	remaining: 17.2s
1200:	learn: 67.7119280	total: 24.5s	remaining: 15.2s
1300:	learn: 66.6703414	total: 26.6s	remaining: 13.2s
1400:	learn: 65.6692110	total: 28.4s	remaining: 11.1s
1500:	learn: 64.6994132	total: 30.6s	remaining: 9.11s
1600:	learn: 63.8019179	total: 32.7s	remaining: 7.07s
1700:	learn: 62.9597480	total: 35s	remain

[I 2023-11-07 09:13:11,829] Trial 79 finished with value: 79.80094251291628 and parameters: {'iterations': 1947, 'learning_rate': 0.008640019826943353, 'depth': 8, 'min_data_in_leaf': 74, 'l2_leaf_reg': 4, 'bagging_temperature': 0.8963094685408294, 'random_strength': 0.6553407063339308, 'border_count': 47, 'rsm': 0.44220223381514345}. Best is trial 55 with value: 72.65112909096902.


1946:	learn: 60.9585103	total: 39.7s	remaining: 0us


In [9]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_b.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1480, 'learning_rate': 0.004154859937303772, 'depth': 1, 'colsample_bylevel': 0.8980589862889544, 'min_data_in_leaf': 11, 'l2_leaf_reg': 7, 'has-time': True, 'bagging_temperature': 0.8588447919947089, 'random_strength': 0.5000970078140154, 'border_count': 358, 'rsm': 0.31318686704086646, 'nan_mode': 'min'}
82.32735192987839
